In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Load data
train = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv', index_col='id')
test  = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv', index_col='id')

X = train.drop('y', axis=1)
y = train['y']

# Split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Categorical & numerical columns
cat_cols = X.select_dtypes(include='object').columns
num_cols = X.select_dtypes(exclude='object').columns

# Preprocessing
preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('num', 'passthrough', num_cols)
    ]
)

# Model
model = Pipeline(
    steps=[
        ('preprocess', preprocess),
        ('clf', RandomForestClassifier(
            n_estimators=200,
            random_state=42,
            n_jobs=-1
        ))
    ]
)

# Train
model.fit(X_train, y_train)

# Evaluate
y_valid_pred = model.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_valid, y_valid_pred)
print("Validation ROC-AUC:", auc)

# Predict test
test_pred = model.predict_proba(test)[:, 1]

# Submission
submission = pd.DataFrame({
    'id': test.index,
    'y': test_pred
})
submission.to_csv('submission.csv', index=False)

print("submission.csv saved")


Validation ROC-AUC: 0.9611612571765439
submission.csv saved
